In [24]:
import bs4
import lxml
import pandas as pd
import urllib
import re

from urllib import request

base_url = "https://howlongtobeat.com/game/"

In [29]:
def get_title(page):
    title = page.find("title").text
    title = title[12:].split('?')[0]
    return title

def get_rating(page):
    rating_tag = page.find("a", attrs={"class": "text_primary", "href": re.compile("/game/[0-9]+/reviews")})
    if rating_tag is None:
        return pd.NA

    rating = rating_tag.text.split("%")[0]
    return rating

def get_platform(page):
    classes = page.find_all("div","GameSummary_profile_info__HZFQu GameSummary_medium___r_ia")
    for classe in classes:
        if "Platform" in classe.find("strong").text:
            platform = classe.text
            platform = platform.split(":")[1][1:]
            platform = platform.split(', ')
            return platform
    return pd.NA # pas de plateforme renseignee

def get_genre(page):
    classes = page.find_all("div","GameSummary_profile_info__HZFQu GameSummary_medium___r_ia")
    for classe in classes:
        if "Genre" in classe.find("strong").text:
            genre = classe.text
            genre = genre.split(":")[1][1:]
            genre = genre.split(', ')
            return genre
    return pd.NA # pas de genre renseigne

def get_date(page):
    classes = page.find_all("div","GameSummary_profile_info__HZFQu")
    for classe in classes:
        description = classe.find("strong")
        if description != None:
            if "NA" in description.text:
                date = classe.text
                date = date.split(":")[1][1:]
                return int(date[-4:])
    return pd.NA # pas de date renseignee

def get_time(page):
    table = page.find("table","GameTimeTable_game_main_table__7uN3H")
    if table != None:
        colonne_main_story = table.find("tr","spreadsheet")
        time_main_story = colonne_main_story.find_all("td")[2]
        time_main_story = time_main_story.text
        time_main_story = time_main_story.split(" ")
        if len(time_main_story)>1:
            heures = float(time_main_story[0][:-1])
            minutes = float(time_main_story[1][:-1])
            time = heures + minutes*1/60
        else:
            time = time_main_story[0][:-1]

        return float(time)
    else:
        return pd.NA # temps non renseigné
    
def get_memoire_vive(page):
    url_steam = get_url_steam(page)
    page_steam = get_page(url_steam)
    uls = page.find_all("ul")
    for ul in uls:
        description = ul.find("strong")
        if description != None:
            if "Recommandée" == description.text:
                    lis = ul.find_all("li")
                    for li in lis:
                        if "Mémoire vive :" == li.find("strong").text:
                            memoire_vive = li.text.split(":")[1]
                            memoire_vive = memoire_vive.split(" ")[0]
                            return int(memoire_vive)
    return pd.NA

def get_espace_disque(page):
    url_steam = get_url_steam(page)
    page_steam = get_page(url_steam)
    uls = page.find_all("ul")
    for ul in uls:
        description = ul.find("strong")
        if description != None:
            if "Recommandée" == description.text:
                    lis = ul.find_all("li")
                    for li in lis:
                        if "Espace disque :" == li.find("strong").text:
                            espace_disque = li.text.split(":")[1]
                            espace_disque = espace_disque.split(" ")[0]
                            return int(espace_disque)
    return pd.NA

def get_pourcentage_pos(page):
    url_steam = get_url_steam(page)
    page_steam = get_page(url_steam)
    div = page_steam.find("div","user_reviews_summary_row")
    span = div.find("span","nonresponsive_hidden responsive_reviewdesc")
    texte = span.text[2:-74]
    texte = texte.split(" ")
    nb_eval = texte[2]
    nb_eval.replace(',','')
    pourcentage = texte[0]
    pourcentage = pourcentage[:-4]
    return pourcentage, nb_eval





In [26]:
def get_url_steam(page):
    a = page.find("a", "GameSummary_steam_text__Fe9Uv")
    if a is None:
        return

    return a["href"]


def get_page(url):
    try:
        request_text = request.urlopen(url).read()
        page = bs4.BeautifulSoup(request_text, "lxml")
    except urllib.error.HTTPError:
        return

    return page


def get_price(steam_page):
    div = page.find("div", attrs={"class":"game_purchase_price", "data-price-final":re.compile('[0-9]+')})

    if div is None:
        return pd.NA

    return div.text

In [30]:
df = pd.DataFrame(columns=["title","rating","platform","avg_time","genre","date"])

# 108288
for i in range(40000, 40030):
    try:
        request_text = request.urlopen(base_url + str(i)).read()
        page = bs4.BeautifulSoup(request_text, "lxml")
    except urllib.error.HTTPError:
        continue
    df2 = pd.DataFrame({"title":get_title(page),"rating":get_rating(page),
                    "platform":[get_platform(page)],"avg_time":get_time(page),
                    "genre":[get_genre(page)],"date":get_date(page)})
    print(get_pourcentage_pos(page))
    
    df = pd.concat([df,df2],ignore_index=True)

df

texte ['\t\t\t\t\t\t']


IndexError: list index out of range